# Project 3
## Scrape data from CDC Opioid Prescription website

In [1]:
# Import pandas
import pandas as pd
import os
import glob

[CDC Opioid Prescription Site](https://www.cdc.gov/drugoverdose/rxrate-maps/index.html). Inspect the page to identify which elements to scrape.

In [2]:
basestateUrl = 'https://www.cdc.gov/drugoverdose/rxrate-maps/state'
basecountyUrl = 'https://www.cdc.gov/drugoverdose/rxrate-maps/county'

In [3]:
#  all years is all years available on the site,  years is the range of data to pull for the website
# allyears = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [4]:
# scrape state level table info and write to a csv
for year in years:
    url = f'{basestateUrl}{year}.html'
    # print(url)
    tablelist = pd.read_html(url)
    tabledf = tablelist[0]
    tabledf['Year'] = year
    # tabledf[f'{year} ODR per 100'] = tabledf['Opioid Dispensing Rate per 100']
    # print(tabledf.head(5))
    # print(len(tabledf))
    tabledf.to_csv(f'Resources/DispenseRate/DRState{year}.csv', index = False)

In [ ]:
# scrape county level table info and write to a csv - not using county level data keep just in case
# for year in years:
#     url = f'{basecountyUrl}{year}.html'
#     # print(url)
#     tablelist = pd.read_html(url)
#     tabledf = tablelist[0]
#     tabledf['Year'] = year
#     # print(tabledf.head(5))
#     # print(len(tabledf))
#     tabledf.to_csv(f'Resources/DispenseRate/County/DRCounty{year}.csv', index = False)

In [5]:
# get list of csvs in State directory
# listCSVs = os.listdir('Resources/DispenseRate/State')
# listCSVs
allFiles = os.listdir("Resources/DispenseRate")    
csvFiles = list(filter(lambda f: f.endswith('.csv'), allFiles))
csvFiles


['DRState2013.csv',
 'DRState2014.csv',
 'DRState2015.csv',
 'DRState2016.csv',
 'DRState2017.csv',
 'DRState2018.csv',
 'DRState2019.csv']

In [6]:
# create list of dataframes
dfs = []
for csvFile in csvFiles:
    dfs.append(pd.read_csv(f'Resources/DispenseRate/{csvFile}'))

In [7]:
#  check some dfs in list
print(dfs[3].head(2))
dfs[4].head(2)

     State State Abbreviation  Opioid Dispensing Rate per 100  Year
0  Alabama                 AL                           121.0  2016
1   Alaska                 AK                            58.9  2016


,State,Abbreviation,Opioid Dispensing Rate per 100,Year
0,United States,US,59.0,2017
1,Alaska,AK,52.0,2017


In [8]:
# create new list of dataframes with standardized column names (changes between 2016-2017)
dffy = []
for dff in dfs:
   dfff = dff.rename(columns = {'State Abbreviation': 'Abbreviation'})
   dffy.append(dfff)


In [9]:
#  check some dfs in list
print(dffy[3].head(2))
dffy[4].head(2)

     State Abbreviation  Opioid Dispensing Rate per 100  Year
0  Alabama           AL                           121.0  2016
1   Alaska           AK                            58.9  2016


,State,Abbreviation,Opioid Dispensing Rate per 100,Year
0,United States,US,59.0,2017
1,Alaska,AK,52.0,2017


In [10]:
# check to see that info was written into dffy list
len(dffy)

7

In [11]:
# dlite list will be used to create ODisp2013-2013.csv 
dlite = []
for d in dfs:
    year = d['Year'].min()
    dd = d.rename(columns = {'Opioid Dispensing Rate per 100':f'{year} ODR per 100'}) 
    dds = dd.rename(columns = {'State Abbreviation': 'Abbreviation'})
    dds2 = dds.drop(columns = ['Year'])
    # print(dds2)
    dlite.append(dds2)
    

In [12]:
# check to see that info was written into dlite list
len(dlite)

7

In [13]:
#  check some dfs in list
print(dfs[3].head(2))
dfs[4].head(2)

     State State Abbreviation  Opioid Dispensing Rate per 100  Year
0  Alabama                 AL                           121.0  2016
1   Alaska                 AK                            58.9  2016


,State,Abbreviation,Opioid Dispensing Rate per 100,Year
0,United States,US,59.0,2017
1,Alaska,AK,52.0,2017


In [14]:

# Join pairs of dfs  
wideFramedf0 = pd.merge(dlite[0],dlite[1], on=['State','Abbreviation'], how = 'outer')
wideFramedf1 = pd.merge(dlite[2],dlite[3], on=['State','Abbreviation'], how = 'outer')
wideFramedf2 = pd.merge(dlite[4],dlite[5], on=['State','Abbreviation'], how = 'outer')
# wideFramedf3 = pd.merge(dlite[6],dlite[7], on=['State','Abbreviation'], how = 'outer')
# wideFramedf4 = pd.merge(dlite[8],dlite[9], on=['State','Abbreviation'], how = 'outer')
# wideFramedf5 = pd.merge(dlite[10],dlite[11], on=['State','Abbreviation'], how = 'outer')
# wideFramedf6 = pd.merge(dlite[12],dlite[13], on=['State','Abbreviation'], how = 'outer')
# join originally written for years 2006-2019


In [15]:
# join quads of dfs
wf0 = pd.merge(wideFramedf0,wideFramedf1, on=['State','Abbreviation'], how = 'outer')
wf1 = pd.merge(wideFramedf2,dlite[6], on=['State','Abbreviation'], how = 'outer')
# wf2 = pd.merge(wideFramedf4,wideFramedf5, on=['State','Abbreviation'], how = 'outer')
# join originally written for years 2006-2019

In [16]:
# extra join step needed for joining years 2006-2019
# wff0 = pd.merge(wf0,wf1, on=['State','Abbreviation'], how = 'outer')
# # wff1 = pd.merge(wf2, wideFramedf6, on=['State','Abbreviation'], how = 'outer')

In [17]:
# final join of all dfs
wideframe = pd.merge(wf0, wf1, on=['State','Abbreviation'], how = 'outer')
print(len(wideframe)) # all 50 states, District of Columbia and overall US
wideframe.to_csv('Resources/ODisp2013_2019.csv', index = False)
wideframe.head(2)


52


,State,Abbreviation,2013 ODR per 100,2014 ODR per 100,2015 ODR per 100,2016 ODR per 100,2017 ODR per 100,2018 ODR per 100,2019 ODR per 100
0,Alabama,AL,142.4,135.2,125.0,121.0,108.8,97.5,85.8
1,Alaska,AK,63.7,62.7,60.8,58.9,52.0,44.9,39.1


In [34]:
# update table to prep for transposing in next step
wf = wideframe.drop(columns = ['State'])
wff = wf.rename(columns = {'Abbreviation':'Opioid_Dispense_Rates_per_Year','2013 ODR per 100':'2013','2014 ODR per 100':'2014','2015 ODR per 100':'2015','2016 ODR per 100':'2016','2017 ODR per 100':'2017','2018 ODR per 100':'2018','2019 ODR per 100':'2019'})

wfff = wff.set_index('Opioid_Dispense_Rates_per_Year')# to prepare for next step - transpose table


In [53]:
# transpose table into format for barchart use and save to csv
wideframetrans = pd.DataFrame(wfff).transpose().reset_index()
# wideframetrans
# wft = wideframetrans.reset_index()
wft= wideframetrans.rename(columns = {'index': 'Year'})
wft
wft.to_csv('Resources/ODispTransposed2013_2019.csv', index = False)
wft.head(2)

Opioid_Dispense_Rates_per_Year,Year,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,TN,TX,UT,VT,VA,WA,WV,WI,WY,US
0,2013,142.4,63.7,80.4,120.9,54.4,71.2,67.4,92.7,41.1,...,127.1,70.0,82.1,52.2,76.6,75.2,129.0,73.8,81.5,NaN
1,2014,135.2,62.7,79.7,123.2,52.7,69.6,66.0,91.0,40.1,...,121.3,67.0,78.8,50.4,73.5,74.2,126.4,71.9,80.9,NaN


In [54]:
# create json file from csv - json will be used for interactive barcharts on the website
df = pd.read_csv (r'Resources\ODispTransposed2013_2019.csv')
df.to_json (r'BarCharts\ODispTransposed2013_2019.json')

In [ ]:
#create general use csv for potential use in other visualizations
bigFrame = pd.concat(dffy, ignore_index = True)
bigFrame.to_csv('Resources/ODisp2013_2019bigframe.csv', index = False)
bigFrame.head(2)


In [ ]:
# get column names
bigFrame.columns